# Исследование базы данных сервиса по чтению книг по подписке.

# Описание проекта 
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.


# Задачи исследования
1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, исключая из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa
pd.set_option('display.max_colwidth', 1000)

In [2]:
# устанавливаем параметры
db_config = {
    'user': 'user', # имя пользователя
    'pwd': 'PASWORD', # пароль
    'host': 'host',
    'port': 1234, # порт подключения
    'db': 'db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

## Первичный осмотр данных

In [5]:
query_books = '''SELECT * FROM books LIMIT 5'''
query_authors = '''SELECT * FROM authors LIMIT 5'''
query_ratings = '''SELECT * FROM ratings LIMIT 5'''
query_reviews = '''SELECT * FROM reviews LIMIT 5'''
query_publishers = '''SELECT * FROM publishers LIMIT 5'''

### Таблица `books`

In [6]:
get_sql_data(query_books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица `authors`

In [7]:
get_sql_data(query_authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица `ratings`

In [8]:
get_sql_data(query_ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица `reviews`

In [9]:
get_sql_data(query_reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


### Таблица `publishers`

In [10]:
get_sql_data(query_publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Вывод:**
Данные соответвуют описанию, таблицы корректы.

## Решение задач

### Задача 1. Посчитать, сколько книг вышло после 1 января 2000 года

In [11]:
query = '''
        select  count(*) AS cnt_books
        from books
        where publication_date > '2000-01-01'
        '''

In [12]:
get_sql_data(query)

,cnt_books
0,819


**Вывод:** 

C 1 января 2000 года было опубликовано 819 книг

### Задача 2. Для каждой книги посчитать количество обзоров и среднюю оценку

In [19]:
query = '''
        SELECT DISTINCT(b.title),
                round(AVG(rating) OVER (PARTITION BY r.book_id),2) AS rt,
                cnt.cnt_rev
        FROM ratings r 
        INNER JOIN books b ON r.book_id = b.book_id 
        LEFT JOIN (
                SELECT DISTINCT(book_id) AS b_id,
                        count(review_id) OVER (PARTITION BY book_id) AS cnt_rev
                FROM reviews r ) AS cnt
        ON r.book_id = cnt.b_id
        ORDER BY cnt_rev DESC, rt DESC
        LIMIT 10;
        '''

In [20]:
get_sql_data(query)

,title,rt,cnt_rev
0,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,NaN
1,Essential Tales and Poems,4.00,NaN
2,Leonardo's Notebooks,4.00,NaN
3,Disney's Beauty and the Beast (A Little Golden Book),4.00,NaN
4,Anne Rice's The Vampire Lestat: A Graphic Novel,3.67,NaN
5,The Natural Way to Draw,3.00,NaN
6,Twilight (Twilight #1),3.66,7.0
7,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),4.41,6.0
8,Harry Potter and the Chamber of Secrets (Harry Potter #2),4.29,6.0
9,The Book Thief,4.26,6.0


In [26]:
query = '''
        SELECT DISTINCT(b.title),
                round(AVG(rating) OVER (PARTITION BY r.book_id),2) AS rt,
                cnt.cnt_rev
        FROM ratings r 
        INNER JOIN books b ON r.book_id = b.book_id 
        LEFT JOIN (
                SELECT DISTINCT(book_id) AS b_id,
                        count(review_id) OVER (PARTITION BY book_id) AS cnt_rev
                FROM reviews r ) AS cnt
        ON r.book_id = cnt.b_id
        ORDER BY rt DESC, cnt_rev DESC;
        '''

In [27]:
get_sql_data(query)

,title,rt,cnt_rev
0,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,NaN
1,A Dirty Job (Grim Reaper #1),5.00,4.0
2,Moneyball: The Art of Winning an Unfair Game,5.00,3.0
3,School's Out—Forever (Maximum Ride #2),5.00,3.0
4,A Fistful of Charms (The Hollows #4),5.00,2.0
...,...,...,...
995,The World Is Flat: A Brief History of the Twenty-first Century,2.25,3.0
996,Drowning Ruth,2.00,3.0
997,His Excellency: George Washington,2.00,2.0
998,Junky,2.00,2.0


**Вывод:**

Не все книги имеют обзоры. 

Наибольшее количество обзоров на книгу "Twilight (Twilight #1)" - 7, оценка при этом 3.66

Книга "A Dirty Job (Grim Reaper #1)" с 4-мя обзорами и меет оценку 5 

Самая низкий рейтинг - 1.5 имеет книга "Harvesting the Heart" и на неё написано 2 обзора.


### Задача 3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, исключая из анализа брошюры

In [28]:
query = '''
        SELECT p.publisher,
            m.cnt_book
        FROM publishers p	
        INNER JOIN (
                SELECT DISTINCT(publisher_id) AS pbl,
                    count(book_id) OVER (PARTITION BY publisher_id) AS cnt_book
                FROM books b 
                WHERE num_pages > 50
                ORDER BY cnt_book DESC 
            ) AS m
        ON p.publisher_id = m.pbl
        LIMIT 1
        '''

In [18]:
get_sql_data(query)

,publisher
0,Penguin Books


**Вывод:**

Больше всего книг было выпущено издательством "Penguin Books", они выпустили 42 книги.

### Задача 4. Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [31]:
query = '''
    WITH max_avg_author AS (
                    SELECT DISTINCT author_id AS aut_id,
                        round(avg(b_rt.avg_rt) OVER (PARTITION BY author_id), 2) AS avg_rt_aut
                    FROM books b 
                    INNER JOIN (
                            SELECT book_id AS id_b,
                                round(avg(rating), 2) AS avg_rt
                            FROM ratings r 
                            GROUP BY book_id 
                            HAVING count(rating_id) >= 50 
                            ) AS b_rt
                    ON b.book_id = b_rt.id_b
                    ORDER BY avg_rt_aut DESC
                    LIMIT 1)
                    
        SELECT author,
            max_avg_author.avg_rt_aut
        FROM author a 
        INNER JOIN max_avg_author
        ON a.author_id = max_avg_author.aut_id
        '''

In [32]:
get_sql_data(query)

,author,avg_rt_aut
0,J.K. Rowling/Mary GrandPré,4.29


**Вывод:** 

Самую высокую средний рейтинг за свои книги (оценок свыше 50) получила J.K. Rowling/Mary GrandPré, средняя оценка её книг составляет 4.29.

### Задача 5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [33]:
query = '''
        select round(avg(many_cnt.cnt))
        from (
            select r.username,
                count(r.review_id) as cnt
            from reviews r inner join 
                    (
                        select username as us,
                    count(rating_id) as rt
                    from ratings r 
                    group by r.username 
                    having count(rating_id) > 48) as users
                    on r.username = users.us
        group by r.username
        ) as many_cnt'''

In [34]:
get_sql_data(query)

,round
0,24.0


**Вывод:**

Среди пользователей, поставивиших больше 48 оценок, в среденм написано по 24 обзора

# Общий вывод
1. С 1 января 2000 года было выпубщено и опубликовано в приложении 819 книг;
2. Самой популярной книгой для написания обзоров была "Twilight" (part 1);
3. Издательство "Penguin Books" выпустило больше всего опубликованных книг в приложении с содержанием больше 50 страниц;
4. J.K. Rowling/Mary GrandPré самый высоко оценимый автор пользователями с более 50 поставленными оценками;
5. Пользователи, поставившие больше 48 оценок, опубликовали в среднем по 24 обзоры на книги.

**Рекомендации:**
1. Рекомендуется обратить внимание на книги с 2-мя обзорами и, возможно, завышенными оценками;
2. Рекомендуется пополнить базу книг, т.к. 1000 книг мало (в том же Google Play Книги более 250 тыс.), не каждый пользователь сможет найти, что ему "по душе";
3. Рекомендуется мотивировать пользователей писать отзывы, тогда качество оценки будет лучше.
